In [8]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
movies = pd.read_csv('movies.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv', low_memory=False)

#print(movies.head())


final_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
#print(final_dataset.head(10))

final_dataset.fillna(0,inplace=True)
#print(final_dataset.head())

no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')

#print(no_user_voted.head())

# f,ax = plt.subplots(1,1,figsize=(16,4))
# # ratings['rating'].plot(kind='hist')
# plt.scatter(no_user_voted.index,no_user_voted,color='mediumseagreen')
# plt.axhline(y=10,color='r')
# plt.xlabel('MovieId')
# plt.ylabel('No. of users voted')
# plt.show()

final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 10].index,:]

#f,ax = plt.subplots(1,1,figsize=(16,4))
# plt.scatter(no_movies_voted.index,no_movies_voted,color='mediumseagreen')
# plt.axhline(y=50,color='r')
# plt.xlabel('UserId')
# plt.ylabel('No. of votes by user')
# plt.show()

final_dataset=final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
# print(final_dataset.head())

# sample = np.array([[0,0,3,0,0],[4,0,0,0,2],[0,0,0,0,1]])
# sparsity = 1.0 - ( np.count_nonzero(sample) / float(sample.size) )
# #print(sparsity)
# csr_sample = csr_matrix(sample)
# print(csr_sample)

csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)
# print(csr_data)
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 10
    movie_list = movies[movies['title'].str.contains(movie_name)]  
    if len(movie_list): 
        print (movie_list       )
        movie_idx= movie_list.iloc[0]['movieId']
        print(movie_idx)
        movie_idx = final_dataset[final_dataset['movieId'] == movie_idx].index[0]
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = final_dataset.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "No movies found. Please check your input"

if __name__ == "__main__":
    print(get_movie_recommendation('Iron Man'))

      movieId              title                                 genres
7714    59315    Iron Man (2008)                Action|Adventure|Sci-Fi
8542    77561  Iron Man 2 (2010)  Action|Adventure|Sci-Fi|Thriller|IMAX
9658   102125  Iron Man 3 (2013)            Action|Sci-Fi|Thriller|IMAX
59315
                           Title  Distance
1             Ratatouille (2007)  0.414882
2                Watchmen (2009)  0.412024
3                     300 (2007)  0.405918
4               Inception (2010)  0.401255
5           Batman Begins (2005)  0.396326
6   Bourne Ultimatum, The (2007)  0.384028
7           Avengers, The (2012)  0.383603
8               Star Trek (2009)  0.347590
9           Casino Royale (2006)  0.336973
10       Dark Knight, The (2008)  0.318392
